In [5]:
# Import libraries
from langchain_community.graphs import Neo4jGraph
from graph_rag_query import GraphRAGQuery
import json

In [7]:
# Setup Neo4j connection
NEO4J_URI = "neo4j+s://0c367113.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "gTO1K567hBLzkRdUAhhEb-UqvBjz0i3ckV3M9v_-Nio"

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

print("Kết nối Neo4j thành công!")

Kết nối Neo4j thành công!


In [8]:
# Initialize GraphRAGQuery
rag_query = GraphRAGQuery(graph)
print("GraphRAGQuery đã sẵn sàng!")
print("Indexes đã được tạo để tối ưu performance")

GraphRAGQuery đã sẵn sàng!
Indexes đã được tạo để tối ưu performance


## Test 1: Get Statistics

In [9]:
# Lấy thống kê tổng quan
stats = rag_query.get_relationship_statistics()

print("=== THỐNG KÊ GRAPH ===")
print(f"Tổng số nodes: {stats['total_nodes']:,}")
print(f"Tổng số relationships: {stats['total_relationships']:,}")
print(f"\nTop 10 relationship types:")
for i, rel_type in enumerate(stats['relationship_types'][:10], 1):
    print(f"  {i:2d}. {rel_type['relationship_type']}: {rel_type['count']:,}")

=== THỐNG KÊ GRAPH ===
Tổng số nodes: 1,499
Tổng số relationships: 1,172

Top 10 relationship types:
   1. LÃNH_ĐẠO: 47
   2. THÀNH_PHẦN: 35
   3. LOCATED_IN: 34
   4. THAM_GIA: 30
   5. HOẠT_ĐỘNG: 26
   6. THÀNH_VIÊN: 24
   7. THÀNH_LẬP: 20
   8. KHÁNG_CHIẾN: 17
   9. PART_OF: 16
  10. LEADER_OF: 15


## Test 2: Most Connected Entities (Hubs)

In [10]:
# Tìm các entities quan trọng nhất (có nhiều connections)
hubs = rag_query.get_most_connected_entities(limit=15)

print("=== TOP 15 ENTITIES QUAN TRỌNG NHẤT ===")
for i, hub in enumerate(hubs, 1):
    print(f"{i:2d}. {hub['name']} ({hub['type']})")
    print(f"    Connections: {hub['connection_count']}")
    if hub['description']:
        print(f"    Mô tả: {hub['description'][:80]}...")
    print()

=== TOP 15 ENTITIES QUAN TRỌNG NHẤT ===
 1. Việt Nam (LOCATION)
    Connections: 96
    Mô tả: Quốc gia...

 2. Pháp (LOCATION)
    Connections: 93
    Mô tả: Quốc gia...

 3. Chính phủ (ORGANIZATION)
    Connections: 87
    Mô tả: Chính quyền...

 4. Pháp (ORGANIZATION)
    Connections: 87
    Mô tả: Quân đội Pháp...

 5. Hồ Chí Minh (PERSON)
    Connections: 79
    Mô tả: Chủ tịch nước...

 6. Việt Nam Dân chủ Cộng hòa (ORGANIZATION)
    Connections: 49
    Mô tả: Nhà nước...

 7. quân Pháp (ORGANIZATION)
    Connections: 46

 8. Nam Bộ (LOCATION)
    Connections: 37
    Mô tả: Khu vực miền Nam Việt Nam...

 9. Hà Nội (LOCATION)
    Connections: 29
    Mô tả: Thành phố...

10. Thực dân Pháp (ORGANIZATION)
    Connections: 26
    Mô tả: Kẻ thù...

11. Trung ương Đảng (ORGANIZATION)
    Connections: 24
    Mô tả: Trung ương Đảng Việt Minh...

12. Liên khu IV (LOCATION)
    Connections: 21
    Mô tả: Tên một liên khu...

13. Việt Bắc (LOCATION)
    Connections: 20
    Mô tả: Vùng địa lý

## Test 3: Search Entities by Name

In [12]:
# Tìm kiếm entities
search_terms = ["Hồ Chí Minh", "Việt Nam", "Pháp", "chiến dịch"]

for term in search_terms:
    results = rag_query.search_entities_by_name(term, limit=5)
    print(f"\nTìm kiếm: '{term}' - Tìm thấy {len(results)} kết quả")
    for r in results:
        print(f"  - {r['name']} ({r['type']})")
        if r['description']:
            print(f"    {r['description'][:60]}...")


Tìm kiếm: 'Hồ Chí Minh' - Tìm thấy 5 kết quả
  - Hồ Chí Minh (PERSON)
    Chủ tịch nước...
  - Chủ tịch Hồ Chí Minh (PERSON)
    Ban bố sắc lệnh số 75...
  - HỒ Chí Minh (PERSON)
  - Bảo tàng Hồ Chí Minh (LOCATION)
    Địa điểm...
  - Chính phủ Hồ Chí Minh (ORGANIZATION)

Tìm kiếm: 'Việt Nam' - Tìm thấy 5 kết quả
  - VIỆN HÀN LÂM KHOA HỌC XÃ HỘI VIỆT NAM (ORGANIZATION)
  - Việt Nam (LOCATION)
    Quốc gia...
  - Việt Nam quốc sử khảo (EVENT)
    Sách...
  - Viện Hàn lâm Khoa học xã hội Việt Nam (ORGANIZATION)
    Trụ sở của Viện Sử học...
  - Tập I Lịch sử Việt Nam (EVENT)
    Xuất bản năm 1971...

Tìm kiếm: 'Pháp' - Tìm thấy 5 kết quả
  - Bản án chế độ thực dân Pháp (EVENT)
    Sách...
  - Tổng chỉ huy lực lượng lục quân Pháp (ORGANIZATION)
  - Bọn thực dân Pháp (ORGANIZATION)
    Quân đội Pháp...
  - Pháp (LOCATION)
    Quốc gia...
  - quân Pháp (ORGANIZATION)

Tìm kiếm: 'chiến dịch' - Tìm thấy 3 kết quả
  - Chiến dịch Biên giới (EVENT)
    Chiến dịch quân sự...
  - Bộ chi huy chiến

## Test 4: Get Entity by Exact Name

In [13]:
# Tìm entity cụ thể (thay tên phù hợp với data của bạn)
entity_name = "Hồ Chí Minh"  # Thay đổi tên này

entity = rag_query.get_entity_by_name(entity_name)

if entity:
    print(f"=== ENTITY: {entity['name']} ===")
    print(f"Type: {entity['type']}")
    print(f"Description: {entity['description']}")
    print(f"First seen: Chapter {entity['chapter']}, Page {entity['page']}")
else:
    print(f"Không tìm thấy entity '{entity_name}'")

=== ENTITY: Hồ Chí Minh ===
Type: PERSON
Description: Chủ tịch nước
First seen: Chapter None, Page 14


## Test 5: Get Entity Neighborhood

In [14]:
# Lấy neighborhood của entity (entities liên quan trực tiếp)
if entity:
    neighborhood = rag_query.get_entity_neighborhood(entity_name)
    
    print(f"=== NEIGHBORHOOD: {entity_name} ===")
    print(f"\nConnected entities: {len(neighborhood['connected_entities'])}")
    print(f"Relationship types: {len(neighborhood['relationships'])}")
    
    print(f"\nRelationships:")
    for rel_type, connections in neighborhood['relationships'].items():
        print(f"\n  {rel_type}: ({len(connections)} connections)")
        for conn in connections[:3]:  # Show first 3
            direction_arrow = "→" if conn['direction'] == 'outgoing' else "←"
            print(f"    {direction_arrow} {conn['entity']} ({conn['entity_type']})")
            if conn['description']:
                print(f"       {conn['description'][:60]}...")
        if len(connections) > 3:
            print(f"    ... và {len(connections) - 3} connections khác")

=== NEIGHBORHOOD: Hồ Chí Minh ===

Connected entities: 79
Relationship types: 32

Relationships:

  THÀNH_VIÊN: (2 connections)
    → Việt Nam Dân chủ Cộng hòa (ORGANIZATION)
       Xây dựng quan hệ thân thiện...
    → Chính phủ (ORGANIZATION)
       Chủ tịch kiêm Bộ trưởng Bộ Ngoại giao...

  KHÁNG_CHIẾN: (1 connections)
    → Hà Nội (LOCATION)
       Chi đạo kháng chiến ở Hà Nội...

  HOẠT_ĐỘNG: (1 connections)
    → Việt Nam (LOCATION)
       tiếp các đại biểu công thương gia...

  LEADER_OF: (13 connections)
    → Việt Nam (LOCATION)
       Chủ tịch Hồ Chí Minh lãnh đạo Việt Nam...
    → Việt Nam Dân chủ Cộng hòa (ORGANIZATION)
       Chủ tịch nước...
    → Hà Tĩnh (LOCATION)
       Chủ tịch Hồ Chí Minh khen tỉnh Hà Tĩnh...
    ... và 10 connections khác

  THAM_GIA: (1 connections)
    → Đảng (ORGANIZATION)
       Gửi thư căn dặn...

  LEAD: (4 connections)
    → Việt Nam (LOCATION)
       Kêu gọi tạo ra nền văn hóa mới cho Việt Nam...
    → Trung ương Đảng (ORGANIZATION)
       L

## Test 6: Get Related Entities (Multi-hop)

In [15]:
# Tìm entities liên quan gián tiếp (2-hop)
if entity:
    related = rag_query.get_related_entities(entity_name, max_depth=2, limit=20)
    
    print(f"=== RELATED ENTITIES (2-hop): {entity_name} ===")
    print(f"Found {len(related)} related entities\n")
    
    # Group by distance
    by_distance = {}
    for r in related:
        dist = r['distance']
        if dist not in by_distance:
            by_distance[dist] = []
        by_distance[dist].append(r)
    
    for distance in sorted(by_distance.keys()):
        entities = by_distance[distance]
        print(f"\n🔹 Distance {distance}: {len(entities)} entities")
        for ent in entities[:5]:  # Show first 5
            print(f"  - {ent['name']} ({ent['type']})")
            if ent['description']:
                print(f"    {ent['description'][:60]}...")
        if len(entities) > 5:
            print(f"  ... và {len(entities) - 5} entities khác")

=== RELATED ENTITIES (2-hop): Hồ Chí Minh ===
Found 20 related entities


🔹 Distance 1: 20 entities
  - Anh (LOCATION)
    Quốc gia...
  - Bảo Đại (PERSON)
    Vua...
  - Bộ Quốc phòng (ORGANIZATION)
    Bộ phận quân đội...
  - Bộ Quốc phòng (ORGANIZATION)
    Bộ phận quân đội...
  - Bộ Tổng tư lệnh (ORGANIZATION)
    Đơn vị ra mệnh lệnh...
  ... và 15 entities khác


## Test 7: Find Paths Between Entities

In [16]:
# Tìm đường đi giữa 2 entities
source = "Hồ Chí Minh"  # Thay đổi
target = "Việt Nam"     # Thay đổi

paths = rag_query.find_paths_between_entities(source, target, max_depth=4)

if paths:
    print(f"=== PATHS: {source} → {target} ===")
    print(f"Found {len(paths)} path(s)\n")
    
    for i, path in enumerate(paths, 1):
        print(f"\nPath {i} (length: {path['path_length']}):")
        
        # Print nodes
        for j, node in enumerate(path['nodes']):
            print(f"  {j+1}. {node['name']} ({node['type']})")
            
            # Print relationship to next node
            if j < len(path['relationships']):
                rel = path['relationships'][j]
                print(f"     --[{rel['type']}]-->")
                if rel['description']:
                    print(f"     ({rel['description'][:50]}...)")
else:
    print(f"Không tìm thấy path từ '{source}' đến '{target}'")

=== PATHS: Hồ Chí Minh → Việt Nam ===
Found 2 path(s)


Path 1 (length: 1):
  1. Hồ Chí Minh (PERSON)
     --[HOẠT_ĐỘNG]-->
     (tiếp các đại biểu công thương gia...)
  2. Việt Nam (LOCATION)

Path 2 (length: 2):
  1. Hồ Chí Minh (PERSON)
     --[LÃNH_ĐẠO]-->
     (Hồ Chí Minh được bảo vệ tại Việt Bắc...)
  2. Việt Bắc (LOCATION)
     --[PRINT_MONEY]-->
     (In tiền...)
  3. Việt Nam (ORGANIZATION)


## Test 8: Get Entities by Type

In [17]:
# Lấy entities theo từng loại
entity_types = ["PERSON", "LOCATION", "ORGANIZATION", "EVENT", "TIME"]

for etype in entity_types:
    entities = rag_query.get_entities_by_type(etype, limit=10)
    print(f"\n{etype}: {len(entities)} entities (showing 10)")
    for ent in entities[:5]:
        print(f"  - {ent['name']}")
        if ent['description']:
            print(f"    {ent['description'][:60]}...")
    if len(entities) > 5:
        print(f"  ... và {len(entities) - 5} entities khác")


PERSON: 10 entities (showing 10)
  - 300.000 đồng bào Sài Gòn
  - Alessandri
    Tên của một người Pháp...
  - André Marie
  - Blum
    Không rõ chức vụ cụ thể...
  - Bollaer
  ... và 5 entities khác

LOCATION: 10 entities (showing 10)
  - ATK
    Vùng kháng chiến...
  - ATK Định Hóa
    Căn cứ địa...
  - An Châu
  - An Dương
  - An Lộc
    Địa điểm...
  ... và 5 entities khác

ORGANIZATION: 10 entities (showing 10)
  - An ninh Liên bang
    Tổ chức...
  - Ban Chấp hành Trung ương Đảng
  - Ban Dân quân
    Ban dân quân cấp khu...
  - Ban Sinh - Lý - Hóa
  - Ban Thanh tra Chính phủ
    Ban Thanh tra Chính phủ...
  ... và 5 entities khác

EVENT: 10 entities (showing 10)
  - 56%
  - 75%
  - Bourgoin
    Kế hoạch Bourgoin...
  - Bản án chế độ thực dân Pháp
    Sách...
  - Bộ Lịch sử Việt Nam 15 tập
    Biên soạn và xuất bản trong khoảng 10 năm gần đây...
  ... và 5 entities khác

TIME: 10 entities (showing 10)
  - 1939
    Năm...
  - 1945
    Năm...
  - 1945-1955
    Thời gian...
  - 1946

## Test 9: Complex Query - Entity Context

Kết hợp nhiều queries để lấy full context của một entity

In [19]:
def get_entity_context(entity_name: str):
    """
    Lấy full context của entity cho RAG
    Bao gồm: entity info, neighborhood, related entities
    """
    # Get entity info
    entity = rag_query.get_entity_by_name(entity_name)
    if not entity:
        return None
    
    # Get neighborhood (1-hop)
    neighborhood = rag_query.get_entity_neighborhood(entity_name)
    
    # Get related entities (2-hop)
    related = rag_query.get_related_entities(entity_name, max_depth=2, limit=30)
    
    return {
        "entity": entity,
        "direct_connections": neighborhood['connected_entities'],
        "relationships": neighborhood['relationships'],
        "related_entities": related,
        "connection_count": len(neighborhood['connected_entities'])
    }

# Test
test_entity = "Hồ Chí Minh"  # Thay đổi
context = get_entity_context(test_entity)

if context:
    print(f"=== FULL CONTEXT: {test_entity} ===")
    print(f"\nEntity Info:")
    print(f"  Type: {context['entity']['type']}")
    print(f"  Description: {context['entity']['description']}")
    print(f"  First seen: Chapter {context['entity']['chapter']}, Page {context['entity']['page']}")
    
    print(f"\nConnections:")
    print(f"  Direct connections: {len(context['direct_connections'])}")
    print(f"  Related entities (2-hop): {len(context['related_entities'])}")
    print(f"  Relationship types: {len(context['relationships'])}")
    
    print(f"\nRelationship Summary:")
    for rel_type, connections in list(context['relationships'].items())[:5]:
        print(f"  - {rel_type}: {len(connections)} connections")
else:
    print(f"Không tìm thấy entity '{test_entity}'")

=== FULL CONTEXT: Hồ Chí Minh ===

Entity Info:
  Type: PERSON
  Description: Chủ tịch nước
  First seen: Chapter None, Page 14

Connections:
  Direct connections: 79
  Related entities (2-hop): 30
  Relationship types: 32

Relationship Summary:
  - THÀNH_VIÊN: 2 connections
  - KHÁNG_CHIẾN: 1 connections
  - HOẠT_ĐỘNG: 1 connections
  - LEADER_OF: 13 connections
  - THAM_GIA: 1 connections
